### Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
# import re
# import string
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# # from nltk.tokenize import word_tokenize
# from nltk.data import find
# import numpy as np
# import pandas as pd
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# import pandas as pd
# from langdetect import detect, DetectorFactory, LangDetectException
# from textblob import TextBlob
# from transformers import pipeline
# import torch
# from transformers import BertTokenizer,AutoTokenizer, AutoModel, AutoModelForSequenceClassification
# from keras.preprocessing.text import Tokenizer
# from keras.utils import pad_sequences
# from sklearn.preprocessing import LabelEncoder
# from keras.models import Sequential
# from keras.layers import Embedding, GRU, Dense
# from sklearn.feature_extraction.text import TfidfVectorizer
# # from gensim.models import Word2Vec
# from sklearn.model_selection import train_test_split
# import keras.layers


# Proses Scraping

Proses scraping dilakukan terhadap situs Letterboxd (Salah satu situs komunitas film terbesar yang digunakan untuk membagi ulasan)
- Proses scraping dilakukan terhadap 2 film indonesia (Agak Laen & Vengeance is mine all others pay cash)
- Kedua film tersebut saya pilih karena memiliki average rating yang mirip yaitu sekitar 3.3 - 3.5. Nilai tersebut menandakan adanya polarisasi

In [2]:
headers = {'User-Agent': 'Mozilla/5.0'}

# Pembersihan data rating
def clean_rating(raw):
    if not raw:
        return ''
    return raw.replace('â˜…', '★').replace('Â½', '½').replace('â™', '').strip()


def convert_star_rating(rating_str):
    if not rating_str or 'No rating' in rating_str:
        return None
    stars = rating_str.count('★')
    half_star = '½' in rating_str
    return stars + 0.5 if half_star else stars

# Proses Scraping Utama
def scrape_reviews_from_movie(movie_url, pages=50):
    reviews_data = []

    for page in range(1, pages + 1):
        url = f"{movie_url}page/{page}/"
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to fetch page {page} of {movie_url}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        review_blocks = soup.find_all('li', class_='film-detail')

        print(f"Scraping page {page}/{pages} for {movie_url}...")

        for block in review_blocks:
            # === Review Text ===
            review_p = block.find('div', class_='body-text -prose js-review-body js-collapsible-text')
            review_text = review_p.get_text(strip=True) if review_p else 'No review'
            print("Scraped Review:", review_text)

            # === Username ===
            username_tag = block.find('strong', class_='name')
            username = username_tag['title'] if username_tag and 'title' in username_tag.attrs else 'Anonymous'

            # === Movie Name ===
            movie_tag = soup.find('section', class_='col-17').find('h1')
            movie_name = movie_tag.get_text(strip=True) if movie_tag else 'Unknown Movie'

            # === Rating (clean + convert) ===
            rating_tag = block.find('span', class_=lambda x: x and x.startswith('rating'))
            raw_rating = rating_tag.get_text(strip=True) if rating_tag else ''
            cleaned_rating = clean_rating(raw_rating)
            numeric_rating = convert_star_rating(cleaned_rating)

            reviews_data.append([movie_name, username, review_text, numeric_rating])

        time.sleep(1)

    return reviews_data

# === SCRAPE MULTIPLE MOVIES ===
movie_urls = [
    'https://letterboxd.com/film/agak-laen/reviews/',
    'https://letterboxd.com/film/vengeance-is-mine-all-others-pay-cash/reviews/',
    # Add more movie review base URLs if needed
]

all_reviews = []

for url in movie_urls:
    all_reviews.extend(scrape_reviews_from_movie(url, pages=150))  # 75 pages = ~1500 reviews per movie

print(f"Total reviews scraped: {len(all_reviews)}")

# === SAVE TO CSV ===
with open('letterboxd_reviews_clean.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Movie Name', 'Username', 'Review', 'Rating'])  # Header
    for row in all_reviews:
        writer.writerow(row)

print("Saved to letterboxd_reviews_clean.csv ✅")


Scraping page 1/150 for https://letterboxd.com/film/agak-laen/reviews/...
Scraped Review: Nggak ada yang ngasih tau kalau ini film praktik pesugihan
Scraped Review: THIS IS SO FRESHHHHHH FROM COMEDY HORROR INDONESIA, waktu itu udh lama ga nonton film indonesia krn takut cringe, and this film totally different and UNIQUE, komedinya dapet, horrornya dapet, hikmah dari kisah hidupnya dapet, GOOD JOB🤩
Scraped Review: Menurut saya film ini sangat amat lucu, setiap komedinya memiliki keunikan tersendiri. Benar Benar Agak Laen
Scraped Review: JEGEL MY MAN
Scraped Review: dari awal release udah rewatch sampe 100 kali keknya (lebay)
Scraped Review: Agak Laen bah
Scraped Review: It was a fun and fresh thing to bring haunted house in a movie. But a few jokes made me uncomfortable, perhaps it was not my cup of tea.
Scraped Review: ketawa habis habisannn
Scraped Review: mmm lucu sih tp ga selucu itu jg soalnya jokesnya bbrp ga masuk di aku, lucuan bloopersnya di akhir. honestly jokes ableistnya gan